In [1]:
import os
import pandas as pd
import requests 
from AlphanumericsTeam.data.util import get_aug_oxford_df, filter_df_regions


REPO_ROOT = os.path.abspath(os.path.join(os.path.abspath(''), os.pardir, os.pardir, os.pardir))

In [2]:
#sample input and output

# IP until 30 sep
EXAMPLE_INPUT_FILE = os.path.join(REPO_ROOT, "covid_xprize/validation/data/2020-09-30_historical_ip.csv")
prediction_input_df = pd.read_csv(EXAMPLE_INPUT_FILE,
                                  parse_dates=['Date'],
                                  dtype={"RegionName": str},
                                  encoding="ISO-8859-1")

#print(prediction_input_df)

#prediction from Aug 1 to Aug 4
EXAMPLE_OUTPUT_FILE = os.path.join(REPO_ROOT, "2020-08-01_2020-08-04_predictions_example.csv")
prediction_output_df = pd.read_csv(EXAMPLE_OUTPUT_FILE,
                                   parse_dates=['Date'],
                                   encoding="ISO-8859-1")  


print(prediction_output_df)

     CountryName RegionName       Date  PredictedDailyNewCases  IsSpecialty
0          Aruba        NaN 2020-08-01                0.820071            0
1          Aruba        NaN 2020-08-02                0.872854            0
2          Aruba        NaN 2020-08-03                0.000000            0
3          Aruba        NaN 2020-08-04                0.000000            0
4    Afghanistan        NaN 2020-08-01               80.590128            0
..           ...        ...        ...                     ...          ...
763       Zambia        NaN 2020-08-04              172.532764            0
764     Zimbabwe        NaN 2020-08-01              178.485848            0
765     Zimbabwe        NaN 2020-08-02              142.449493            0
766     Zimbabwe        NaN 2020-08-03               84.436329            0
767     Zimbabwe        NaN 2020-08-04              199.259844            0

[768 rows x 5 columns]


In [3]:
# Input data for training

#DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
#df = pd.read_csv(DATA_URL,
#                 parse_dates=['Date'],
#                 encoding="ISO-8859-1",
#                 dtype={"RegionName": str,
#                        "RegionCode": str},
#                 error_bad_lines=False)

# Has 6 additional columns 
# 'New Cases' 
# 'GeoID' 
# 'Holidays' 
# 'pop_2020' 
# 'area_km2' 
# 'density_perkm2'
df = get_aug_oxford_df() 
df = filter_df_regions(df)

# Final list of 180 countries and 56 regions
assert df.CountryName.unique().size == 180
assert df.RegionName.unique().size == 56 + 1 

In [4]:
print(df.sample(3))
df["DailyChangeConfirmedCases"] = df.groupby(["CountryName", "RegionName"]).ConfirmedCases.diff().fillna(0)
california_df = df[(df.CountryName == "United States") & (df.RegionName == "California")]
california_df[["CountryName", "RegionName", "Date", "ConfirmedCases", "DailyChangeConfirmedCases"]].tail(5)

         CountryName CountryCode   RegionName RegionCode Jurisdiction  \
80842  United States         USA  Connecticut      US_CT  STATE_TOTAL   
77346        Ukraine         UKR                     NaN    NAT_TOTAL   
83382  United States         USA        Idaho      US_ID  STATE_TOTAL   

            Date  C1_School closing  C1_Flag  C2_Workplace closing  C2_Flag  \
80842 2020-10-19                2.0      0.0                   2.0      1.0   
77346 2020-01-19                0.0      NaN                   0.0      NaN   
83382 2020-11-22                2.0      0.0                   2.0      0.0   

       ...  ContainmentHealthIndex  ContainmentHealthIndexForDisplay  \
80842  ...                   66.03                             66.03   
77346  ...                    0.00                              0.00   
83382  ...                   56.73                             56.73   

       EconomicSupportIndex  EconomicSupportIndexForDisplay  NewCases  \
80842                  75.0 

,CountryName,RegionName,Date,ConfirmedCases,DailyChangeConfirmedCases
80187,United States,California,2020-12-19,1842557.0,40362.0
80188,United States,California,2020-12-20,1884033.0,41476.0
80189,United States,California,2020-12-21,1923887.0,39854.0
80190,United States,California,2020-12-22,NaN,0.0
80191,United States,California,2020-12-23,NaN,0.0


In [5]:
# TRAINING
import xprize_predictor
from importlib import reload
reload(xprize_predictor)
from xprize_predictor import XPrizePredictor

DATA_PATH = os.path.join("data", 'OxCGRT_latest_aug.csv')
predictor = XPrizePredictor(None, DATA_PATH)
 
predictor_model = predictor.train()

model_weights_file = "models/trained_model_weights.h5"
if not os.path.exists('models'):
    os.mkdir('models')
predictor_model.save_weights(model_weights_file)

Creating numpy arrays for Keras for each country...
Numpy arrays created
 len geos 234
69988 0.0 57.00170398805195
69988 0.0 57.00170398805195
Trial 0
Epoch 1/100
1969/1969 [==============================] - 6s 3ms/step - loss: 0.4805 - val_loss: 0.0982
Epoch 2/100
1969/1969 [==============================] - 7s 3ms/step - loss: 0.0944 - val_loss: 0.0938
Epoch 3/100
1969/1969 [==============================] - 7s 3ms/step - loss: 0.0928 - val_loss: 0.0923
Epoch 4/100
1969/1969 [==============================] - 6s 3ms/step - loss: 0.0926 - val_loss: 0.0932
Epoch 5/100
1969/1969 [==============================] - 7s 4ms/step - loss: 0.0924 - val_loss: 0.0922
Epoch 6/100
1969/1969 [==============================] - 6s 3ms/step - loss: 0.0922 - val_loss: 0.0929
Epoch 7/100
1969/1969 [==============================] - 6s 3ms/step - loss: 0.0922 - val_loss: 0.0936
Epoch 8/100
1969/1969 [==============================] - 6s 3ms/step - loss: 0.0922 - val_loss: 0.0918
Epoch 9/100
1969/1969 [==

In [6]:
# TESTING AND PREDICTION ---
DATA_PATH = os.path.join("data", 'OxCGRT_latest_aug.csv')
model_weights_file = "models/trained_model_weights.h5"

import xprize_predictor
from importlib import reload
reload(xprize_predictor)

from xprize_predictor import XPrizePredictor

predictor = XPrizePredictor(model_weights_file, DATA_PATH)

NPIS_INPUT_FILE = "../../validation/data/2020-09-30_historical_ip.csv"
start_date = "2020-08-01"
end_date = "2020-08-31"
 
preds_df = predictor.predict(start_date, end_date, NPIS_INPUT_FILE)
preds_df.head()

Start and end date 2020-08-01 00:00:00 2020-08-31 00:00:00
days 31
Index(['CountryName', 'RegionName', 'Date', 'C1_School closing',
       'C2_Workplace closing', 'C3_Cancel public events',
       'C4_Restrictions on gatherings', 'C5_Close public transport',
       'C6_Stay at home requirements', 'C7_Restrictions on internal movement',
       'C8_International travel controls', 'H1_Public information campaigns',
       'H2_Testing policy', 'H3_Contact tracing', 'H6_Facial Coverings',
       'GeoID'],
      dtype='object')
Index(['CountryName', 'RegionName', 'Date', 'PredictedDailyNewCases'], dtype='object')


,CountryName,RegionName,Date,PredictedDailyNewCases
0,Aruba,,2020-08-01,20.676870
1,Aruba,,2020-08-02,23.982245
2,Aruba,,2020-08-03,15.067016
3,Aruba,,2020-08-04,11.619684
4,Aruba,,2020-08-05,15.138159


In [10]:
result = "result"+ "_" + str(start_date) + "_" + end_date + ".csv"
preds_df.to_csv(result, index=False)

In [11]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip ../../validation/data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv
!head predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
Start and end date 2020-08-01 00:00:00 2020-08-04 00:00:00
days 4
Index(['CountryName', 'RegionName', 'Date', 'C1_School closing',
       'C2_Workplace closing', 'C3_Cancel public events',
       'C4_Restrictions on gatherings', 'C5_Close public transport',
       'C6_Stay at home requirements', 'C7_Restrictions on internal movement',
       'C8_International travel controls', 'H1_Public information campaigns',
       'H2_Testing policy', 'H3_Contact tracing', 'H6_Facial Coverings',
       'GeoID'],
      dtype='object')
Index(['CountryName', 'RegionName', 'Date', 'PredictedDailyNewCases'], dtype='object')
Saved predictions to predictions/2020-08-01_2020-08-04.csv
Done!
CountryName,RegionName,Date,PredictedDailyNewCases
Aruba,,2020-08-01,20.676870499063114
Aruba,,2020-08-02,23.982244681924573
Aruba,,2020-08-03,15.067015759656393
Aruba,,2020-08-04,11.619683696970043
Afghanistan,,2020-08-01,244.41638600149582
Afghanistan,,2020-08-02

In [12]:
# Check the pediction file is valid
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

In [13]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

Generating predictions from 2020-08-01 to 2020-08-04...
Start and end date 2020-08-01 00:00:00 2020-08-04 00:00:00
days 4
Index(['CountryName', 'RegionName', 'Date', 'C1_School closing',
       'C2_Workplace closing', 'C3_Cancel public events',
       'C4_Restrictions on gatherings', 'C5_Close public transport',
       'C6_Stay at home requirements', 'C7_Restrictions on internal movement',
       'C8_International travel controls', 'H1_Public information campaigns',
       'H2_Testing policy', 'H3_Contact tracing', 'H6_Facial Coverings',
       'GeoID'],
      dtype='object')
Index(['CountryName', 'RegionName', 'Date', 'PredictedDailyNewCases'], dtype='object')
Saved predictions to predictions/val_4_days.csv
Done!
All good!


In [14]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

Generating predictions from 2021-01-01 to 2021-01-31...
Start and end date 2021-01-01 00:00:00 2021-01-31 00:00:00
days 31
Index(['CountryName', 'RegionName', 'Date', 'C1_School closing',
       'C2_Workplace closing', 'C3_Cancel public events',
       'C4_Restrictions on gatherings', 'C5_Close public transport',
       'C6_Stay at home requirements', 'C7_Restrictions on internal movement',
       'C8_International travel controls', 'H1_Public information campaigns',
       'H2_Testing policy', 'H3_Contact tracing', 'H6_Facial Coverings',
       'GeoID'],
      dtype='object')
Index(['CountryName', 'RegionName', 'Date', 'PredictedDailyNewCases'], dtype='object')
Saved predictions to predictions/val_1_month_future.csv
Done!
All good!
CPU times: user 140 ms, sys: 55.6 ms, total: 195 ms
Wall time: 12.9 s


In [15]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

Start date: 2021-01-04
End date: 2021-07-03


In [16]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

Saved scenario to predictions/180_days_future_scenario.csv


In [17]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")

Generating predictions from 2021-01-04 to 2021-07-03...
Start and end date 2021-01-04 00:00:00 2021-07-03 00:00:00
days 181
^C
Traceback (most recent call last):
  File "predict.py", line 68, in <module>
    predict(args.start_date, args.end_date, args.ip_file, args.output_file)
  File "predict.py", line 35, in predict
    preds_df = predictor.predict(start_date, end_date, path_to_ips_file)
  File "/home/sid/git/covid-xprize/AlphanumericsTeam/predictors/lstm/xprize_predictor.py", line 121, in predict
    back_up = add_features_df(back_up)
  File "/home/sid/git/covid-xprize/AlphanumericsTeam/predictors/lstm/util.py", line 99, in add_features_df
    df[['Holidays', 'density_perkm2']] = df.apply(applyFunc, pop_df=pop_df, axis=1)
  File "/home/sid/git/covid-xprize/venv/lib/python3.7/site-packages/pandas/core/frame.py", line 7547, in apply
    return op.get_result()
  File "/home/sid/git/covid-xprize/venv/lib/python3.7/site-packages/pandas/core/apply.py", line 180, in get_result
    return 

FileNotFoundError: [Errno 2] No such file or directory: 'predictions/val_6_month_future.csv'

In [ ]:

#TESTING/DEBUGGING CODE
import pandas as pd
from util import add_features_df

df = pd.read_csv("../../validation/data/2020-09-30_historical_ip.csv",
                                parse_dates=['Date'],
                                encoding="ISO-8859-1",) 
df["RegionName"] = df["RegionName"].fillna(value="") 
add_features_df(df)